In [1]:
pip install mendeleev


Note: you may need to restart the kernel to use updated packages.


In [2]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
from mendeleev import element

In [3]:
##creating Atom class
class Atom:
    def __init__(self, atType, symbol, userNum, coords, id = -1):
        self.atType = atType
        self.symbol = symbol
        self.userNum = userNum
        self.coords = coords
        self.id = id
        xyz = coords.split()
        self.xcoord = float(xyz[0])
        self.ycoord = float(xyz[1])
        self.zcoord = float(xyz[2])
        
    def __repr__(self):
        return "Atom: id: {}, atType: {}, symbol: {}, userNum: {}, coords: {}".format(self.id, self.atType, self.symbol, self.userNum, self.coords)

In [4]:
##creating Bond class
class Bond:
    def __init__(self, bondAtom1, bondAtom2, bondOrderType, bondOrder):
        self.bondAtom1 = bondAtom1
        self.bondAtom2 = bondAtom2
        self.bondOrderType = bondOrderType
        self.bondOrder = bondOrder
        
    def __repr__(self):
        return " Bond: bondAtom1: {}, bondAtom2: {}, bondOrderType: {}, bondOrder: {}".format(self.bondAtom1, self.bondAtom2, self.bondOrderType, self.bondOrder)

In [5]:
##creating Molecule class
class Molecule:
    
    ##initializer with option to parse c3xml file
    def __init__(self, file=None):
        self.atoms = dict()
        self.bonds = dict()
        if file == None:
            pass
            ##creates empty molecule, atoms and bonds can be added later on
        else:
            tree = ET.parse(file)
            root = tree.getroot()
            for atom in root.iter('atom'):
                id = atom.attrib['id']
                atType = atom.attrib['atType']
                symbol = atom.attrib['symbol']
                userNum = atom.attrib['userNum']
                coords = atom.attrib['cartCoords']
                self.atoms[id] = Atom(atType, symbol, userNum, coords, id)
            for bond in root.iter('bond'):
                id = bond.attrib['id']
                bondAtom1 = bond.attrib['bondAtom1']
                bondAtom2 = bond.attrib['bondAtom2']
                bondOrderType = bond.attrib['bondOrderType']
                bondOrder = bond.attrib['bondOrder']
                self.bonds[id] = Bond(bondAtom1, bondAtom2, bondOrderType, bondOrder)
        self.center = None
        #only for k-hop function
    
    def __repr__(self):
        return "Atoms: {}, Bonds: {}".format(self.atoms.keys(), self.bonds.keys())
    
    def bond_atom1(self, bond):
        return self.bonds[bond].bondAtom1
    
    def bond_atom2(self, bond):
        return self.bonds[bond].bondAtom2
    
    def bond_atom1_info(self, bond):
        return self.atoms[self.bonds[bond].bondAtom1]
    
    def bond_atom2_info(self, bond):
        return self.atoms[self.bonds[bond].bondAtom2]
    
    def plot(self):
        fig = plt.figure(figsize = (20,15))
        ax = plt.axes(projection = '3d')
    
        x = np.array([])
        y = np.array([])
        z = np.array([])
        
        symbol_colors = {'C': 'black', 'H': 'gray', 'O': 'red', 'N': 'blue', "Cl": 'green', 'S': 'yellow', 'P': 'orange', 'F': 'indigo'}
        
        atoms_dict_x = dict()
        atoms_dict_y = dict()
        atoms_dict_z = dict()
        for atom in self.atoms:
            if self.atoms[atom].symbol not in atoms_dict_x.keys():
                atoms_dict_x[self.atoms[atom].symbol] = np.array([])
                atoms_dict_y[self.atoms[atom].symbol] = np.array([])
                atoms_dict_z[self.atoms[atom].symbol] = np.array([])
            else:
                pass
            atoms_dict_x[self.atoms[atom].symbol] = np.append(atoms_dict_x[self.atoms[atom].symbol], self.atoms[atom].xcoord)
            atoms_dict_y[self.atoms[atom].symbol] = np.append(atoms_dict_y[self.atoms[atom].symbol], self.atoms[atom].ycoord)
            atoms_dict_z[self.atoms[atom].symbol] = np.append(atoms_dict_z[self.atoms[atom].symbol], self.atoms[atom].zcoord)
        for symbol in atoms_dict_x.keys():
            if symbol != 'Lp':
                if symbol in symbol_colors.keys():
                    color = symbol_colors[symbol]
                else:
                    color = 'magenta'
                size = 5 * element(symbol).atomic_radius
                ax.scatter(atoms_dict_x[symbol], atoms_dict_y[symbol], atoms_dict_z[symbol], s = size, c = color)
        ##plotting atoms
    
        bond_width = {'0':0,'1': 2, "2":6, "3":12}
        #none, single bond, double bond, triple bond
        for bond in self.bonds:
            xb = np.array([])
            yb = np.array([])
            zb = np.array([])
            xb = np.append(xb, self.atoms[self.bonds[bond].bondAtom1].xcoord)
            xb = np.append(xb, self.atoms[self.bonds[bond].bondAtom2].xcoord)
            yb = np.append(yb, self.atoms[self.bonds[bond].bondAtom1].ycoord)
            yb = np.append(yb, self.atoms[self.bonds[bond].bondAtom2].ycoord)
            zb = np.append(zb, self.atoms[self.bonds[bond].bondAtom1].zcoord)
            zb = np.append(zb, self.atoms[self.bonds[bond].bondAtom2].zcoord)
            ax.plot(xb, yb, zb, linewidth = bond_width[self.bonds[bond].bondOrder], c = 'gray')
        
    
    ##returns atoms & bonds one hop from given atom, used later to construct the k-hop method
    def one_hop(self, atom):
        one_hop = []
        one_hop.append(atom)
        one_hop_bonds = []
        for bond in self.bonds.keys():
            if self.bond_atom1(bond) == atom:
                one_hop.append(self.bond_atom2(bond))
                one_hop_bonds.append(bond)
            elif self.bond_atom2(bond) == atom:
                one_hop.append(self.bond_atom1(bond))
                one_hop_bonds.append(bond)
        return one_hop, one_hop_bonds
    
    ##built this first to use as a guide to build k-hop method
    #def two_hop(self, atom):
        #atoms, bonds = self.one_hop(atom)
        #two_hop = []
        #two_hop_bonds = []
        #for atom in atoms:
            #two_hop.append(atom)
        #for bond in bonds:
            #two_hop_bonds.append(bond)
        #for i in atoms:
            #a, b = self.one_hop(i)
            #for atom in a:
                #if atom not in two_hop:
                    #two_hop.append(atom)
            #for bond in b:
                #if bond not in two_hop_bonds:
                    #two_hop_bonds.append(bond)
        #return two_hop, two_hop_bonds
    
    ##returns molecule with all atoms, bonds in a k-hop neighborhood of an atom
    def k_hop(self, atom, k):
        k_atoms, k_bonds = self.one_hop(atom)
        new_atoms = []
        new_bonds = []
        for iter in range(k-1):
            for i in k_atoms:
                atoms, bonds = self.one_hop(i)
                for atom in atoms:
                    if atom not in k_atoms and atom not in new_atoms:
                        new_atoms.append(atom)
                for bond in bonds:
                    if bond not in k_bonds and bond not in new_bonds:
                        new_bonds.append(bond) 
            for a in new_atoms:
                k_atoms.append(a)
            for b in new_bonds:
                k_bonds.append(b)
            new_atoms = []
            new_bonds = []
        new_molecule = Molecule()
        #added center, center_id 1/19
        new_molecule.center = self.atoms[atom]
        ####################### 
        for atom in k_atoms:
            new_molecule.atoms[atom] = self.atoms[atom]
        for bond in k_bonds:
            new_molecule.bonds[bond] = self.bonds[bond]
        return new_molecule
    
    ###new 1/19
    def get_all_k_hop(self, k):
        neighborhoods = []
        for atom in self.atoms:
            neighborhoods.append(self.k_hop(atom, k))
        return neighborhoods

    ###new 1/21
    def distance(self, bond):
        atom1 = self.bonds[bond].bondAtom1
        atom2 = self.bonds[bond].bondAtom2
        x1 = self.atoms[atom1].xcoord
        y1 = self.atoms[atom1].ycoord
        z1 = self.atoms[atom1].zcoord
        x2 = self.atoms[atom2].xcoord
        y2 = self.atoms[atom2].ycoord
        z2 = self.atoms[atom2].zcoord
        distance = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
        return distance
    
    ##new 1/24
    def distance_atoms(self, atom1, atom2):
        x1 = self.atoms[atom1].xcoord
        y1 = self.atoms[atom1].ycoord
        z1 = self.atoms[atom1].zcoord
        x2 = self.atoms[atom2].xcoord
        y2 = self.atoms[atom2].ycoord
        z2 = self.atoms[atom2].zcoord
        distance = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
        return distance
    
    ##new 1/24
    def no_bond_neighbor(self, atom1):
        closest_distance = 100
        closest = None
        for atom2 in self.atoms:
            one_hop = self.k_hop(atom1,1)
            if atom2 not in one_hop.atoms:
                distance = self.distance_atoms(atom1, atom2)
                if distance < closest_distance:
                    closest_distance = distance
                    closest = atom2
        if closest_distance != 100:
            return closest_distance
        else:
            return -1
    
    ##new 1/29, returns all no_bond_neighbors for all atoms in molecule
    def molecule_no_bond(self):
        distances = []
        for atom in self.atoms:
            if self.no_bond_neighbor(atom) != -1:
                distances.append(self.no_bond_neighbor(atom))
        return distances

In [8]:
test.molecule_no_bond()

[2.0974855674592856,
 2.0965290815774535,
 2.0614541155737616,
 2.124938196021945,
 2.152125408149072,
 2.115239700012743,
 2.3182998315897367,
 2.1163137915699077,
 2.153746135025203,
 2.1243658323191417,
 2.0679999391769823,
 2.115661380514377,
 2.096528467399381,
 2.1251447027437917,
 2.1253204896673816,
 2.095841288385168,
 2.064147760439887,
 2.124693895263268,
 2.4604108749905977,
 1.2175670579729891,
 2.1228495123357187,
 2.114364956768107,
 2.0614541155737616,
 2.0965290815774535,
 2.1156203945415157,
 2.11494821109265,
 2.066301831992606,
 2.1151657405855477,
 2.12213851990015,
 2.096500692916652,
 2.116421639844008,
 2.1156143668610774,
 2.0971134753572587,
 2.1215608112352093,
 2.070537661429997,
 2.1547906488213653,
 2.1156203945415157,
 2.125894650588312,
 2.3305351932097054,
 1.6119932308977758,
 2.117372090328008,
 2.297413913552802,
 2.3243193948175453,
 2.152294507214103,
 2.117204643344615,
 2.3169273975029947,
 2.11494821109265,
 1.605290446806197,
 2.124503060456823

In [9]:
test.no_bond_neighbor('19')

2.0974855674592856

In [7]:
test = Molecule("test.c3xml")
#test19 = test.k_hop('19', 2)
#test19.center
#test.bonds
test.atoms

{'19': Atom: id: 19, atType: C Alkene, symbol: C, userNum: 37, coords: 2.80257 3.51904 2.78315,
 '9': Atom: id: 9, atType: C Alkene, symbol: C, userNum: 1, coords: -5.8038 1.76349 1.81418,
 '10': Atom: id: 10, atType: C Alkene, symbol: C, userNum: 2, coords: -5.58577 0.477388 1.51669,
 '12': Atom: id: 12, atType: C Alkene, symbol: C, userNum: 3, coords: -4.4161 0.105355 0.974627,
 '29': Atom: id: 29, atType: C Alkene, symbol: C, userNum: 40, coords: -0.0147409 -3.89172 1.16861,
 '31': Atom: id: 31, atType: C Alkene, symbol: C, userNum: 41, coords: 0.972577 -4.73654 1.53175,
 '33': Atom: id: 33, atType: C Alkene, symbol: C, userNum: 42, coords: 1.70501 -5.3448 0.585837,
 '35': Atom: id: 35, atType: C Alkene, symbol: C, userNum: 43, coords: 1.47827 -5.12194 -0.718278,
 '37': Atom: id: 37, atType: C Alkene, symbol: C, userNum: 44, coords: 0.505237 -4.26399 -1.09097,
 '40': Atom: id: 40, atType: C Alkene, symbol: C, userNum: 45, coords: -0.776615 -3.38458 2.1623,
 '42': Atom: id: 42, atTyp

In [ ]:
TRASH
def plot(self):
        fig = plt.figure(figsize = (20,15))
        ax = plt.axes(projection = '3d')
    
        x = np.array([])
        y = np.array([])
        z = np.array([])
        
        symbol_colors = {'C': 'black', 'H': 'gray', 'O': 'red', 'N': 'blue', "Cl": 'green', 'S': 'yellow', 'P': 'orange', 'F': 'indigo'}
        
        atoms_dict_x = dict()
        atoms_dict_y = dict()
        atoms_dict_z = dict()
        for atom in self.atoms:
            if self.atoms[atom].symbol not in atoms_dict_x.keys():
                atoms_dict_x[self.atoms[atom].symbol] = np.array([])
                atoms_dict_y[self.atoms[atom].symbol] = np.array([])
                atoms_dict_z[self.atoms[atom].symbol] = np.array([])
            else:
                pass
            atoms_dict_x[self.atoms[atom].symbol] = np.append(atoms_dict_x[self.atoms[atom].symbol], self.atoms[atom].xcoord)
            atoms_dict_y[self.atoms[atom].symbol] = np.append(atoms_dict_y[self.atoms[atom].symbol], self.atoms[atom].ycoord)
            atoms_dict_z[self.atoms[atom].symbol] = np.append(atoms_dict_z[self.atoms[atom].symbol], self.atoms[atom].zcoord)
        for symbol in atoms_dict_x.keys():
            if symbol in symbol_colors.keys():
                color = symbol_colors[symbol]
            else:
                color = 'magenta'
            ax.scatter(atoms_dict_x[symbol], atoms_dict_y[symbol], atoms_dict_z[symbol], s = 200, c = color)
        ##plotting atoms
        #ax.scatter(x,y,z, s = 200, c = 'gray')
    
        for bond in self.bonds:
            xb = np.array([])
            yb = np.array([])
            zb = np.array([])
            xb = np.append(xb, self.atoms[self.bonds[bond].bondAtom1].xcoord)
            xb = np.append(xb, self.atoms[self.bonds[bond].bondAtom2].xcoord)
            yb = np.append(yb, self.atoms[self.bonds[bond].bondAtom1].ycoord)
            yb = np.append(yb, self.atoms[self.bonds[bond].bondAtom2].ycoord)
            zb = np.append(zb, self.atoms[self.bonds[bond].bondAtom1].zcoord)
            zb = np.append(zb, self.atoms[self.bonds[bond].bondAtom2].zcoord)
            #ax.plot(xb, yb, zb)

In [ ]:
OG code
    def plot(self):
        fig = plt.figure(figsize = (20,15))
        ax = plt.axes(projection = '3d')
    
        x = np.array([])
        y = np.array([])
        z = np.array([])
        for atom in self.atoms:
            x = np.append(x, self.atoms[atom].xcoord)
            y = np.append(y, self.atoms[atom].ycoord)
            z = np.append(z, self.atoms[atom].zcoord)
        ##plotting atoms
        ax.scatter(x,y,z, s = 200, c = 'gray')
    
        for bond in self.bonds:
            xb = np.array([])
            yb = np.array([])
            zb = np.array([])
            xb = np.append(xb, self.atoms[self.bonds[bond].bondAtom1].xcoord)
            xb = np.append(xb, self.atoms[self.bonds[bond].bondAtom2].xcoord)
            yb = np.append(yb, self.atoms[self.bonds[bond].bondAtom1].ycoord)
            yb = np.append(yb, self.atoms[self.bonds[bond].bondAtom2].ycoord)
            zb = np.append(zb, self.atoms[self.bonds[bond].bondAtom1].zcoord)
            zb = np.append(zb, self.atoms[self.bonds[bond].bondAtom2].zcoord)
            ax.plot(xb, yb, zb)

In [ ]:
TRASH
def k_hop(self, atom, k):
        list, k_bonds = self.one_hop(atom)
        new_nums = []
        for iter in range(k-1):
            for b in list:
                for a in self.one_hop(b):
                    if a not in list and a not in new_nums:
                        new_nums.append(a)
            for a in new_nums:
                list.append(a)
        return list
    
    
    def k_hop(self, atom, k):
        list, k_bonds = self.one_hop(atom)
        new_nums = []
        new_bonds = []
        for iter in range(k-1):
            for b in list:
                for a,c in self.one_hop(b):
                    if a not in list and a not in new_nums:
                        new_nums.append(a)
                    if c not in k_bonds and c not in new_bonds:
                        new_bonds.append(c)
            for a,c in new_nums, new_bonds:
                list.append(a)
                k_bonds.append(c)
        return list, k_bonds

In [ ]:
TRASH

class Atom:
    def __init__(self, atType, symbol, userNum, coords):
        self.atType = atType
        self.symbol = symbol
        self.userNum = userNum
        self.coords = coords
        
    def __repr__(self):
        return "Atom: atType: {}, symbol: {}, userNum: {}, coords: {}".format(self.atType, self.symbol, self.userNum, self.coords)

class Bond:
    def __init__(self, bondAtom1, bondAtom2, bondOrderType, bondOrder):
        self.bondAtom1 = bondAtom1
        self.bondAtom2 = bondAtom2
        self.bondOrderType = bondOrderType
        self.bondOrder = bondOrder
        
    def __repr__(self):
        return "Bond: bondAtom1: {}, bondAtom2: {}, bondOrderType: {}, bondOrder: {}".format(self.bondAtom1, self.bondAtom2, self.bondOrderType, self.bondOrder)

class Molecule:
    
    atoms = dict()
    bonds = dict()
    
    def __init__(self, file):
        tree = ET.parse(file)
        root = tree.getroot()
        for atom in root.iter('atom'):
            id = atom.attrib['id']
            atType = atom.attrib['atType']
            symbol = atom.attrib['symbol']
            userNum = atom.attrib['userNum']
            coords = atom.attrib['cartCoords']
            self.atoms[id] = Atom(atType, symbol, userNum, coords)
        for bond in root.iter('bond'):
            id = bond.attrib['id']
            bondAtom1 = bond.attrib['bondAtom1']
            bondAtom2 = bond.attrib['bondAtom2']
            bondOrderType = bond.attrib['bondOrderType']
            bondOrder = bond.attrib['bondOrder']
            self.bonds[id] = Bond(bondAtom1, bondAtom2, bondOrderType, bondOrder)
            
    def bond_atom1_info(self, bond):
        return self.atoms[self.bonds[bond].bondAtom1]
    
    def bond_atom2_info(self, bond):
        return self.atoms[self.bonds[bond].bondAtom2]
    
    def bond_atom1(self, bond):
        return self.bonds[bond].bondAtom1
    
    def bond_atom2(self, bond):
        return self.bonds[bond].bondAtom2
    
    def one_hop(self, atom):
        one_hop = []
        one_hop.append(atom)
        one_hop_bonds = []
        for bond in self.bonds.keys():
            if self.bond_atom1(bond) == atom:
                one_hop.append(self.bond_atom2(bond))
                one_hop_bonds.append(bond)
            elif self.bond_atom2(bond) == atom:
                one_hop.append(self.bond_atom1(bond))
                one_hop_bonds.append(bond)
        return one_hop, one_hop_bonds
    
    def two_hop(self, atom):
        atoms, bonds = self.one_hop(atom)
        two_hop = []
        two_hop_bonds = []
        for atom in atoms:
            two_hop.append(atom)
        for bond in bonds:
            two_hop_bonds.append(bond)
        for i in atoms:
            a, b = self.one_hop(i)
            for atom in a:
                if atom not in two_hop:
                    two_hop.append(atom)
            for bond in b:
                if bond not in two_hop_bonds:
                    two_hop_bonds.append(bond)
        return two_hop, two_hop_bonds
        
    def k_hop(self, atom, k):
        k_atoms, k_bonds = self.one_hop(atom)
        new_atoms = []
        new_bonds = []
        for iter in range(k-1):
            for i in k_atoms:
                atoms, bonds = self.one_hop(i)
                for atom in atoms:
                    if atom not in k_atoms and atom not in new_atoms:
                        new_atoms.append(atom)
                for bond in bonds:
                    if bond not in k_bonds and bond not in new_bonds:
                        new_bonds.append(bond) 
            for a in new_atoms:
                k_atoms.append(a)
            for b in new_bonds:
                k_bonds.append(b)
            new_atoms = []
            new_bonds = []
        
        new_molecule = 
    # make it molecule
    # add escriptions